# Model

## Libs

In [ ]:
import gc
import json
import pandas as pd
import unicodedata

## Data

In [ ]:
Data_path = '../data/json/'

In [ ]:
with open(Data_path + 'applicants.json', 'r', encoding='utf-8') as file:
	applicants = json.load(file)

rows = []
for applicant in applicants:
	id = applicant
	informacoes_pessoais = applicants.get(id).get("informacoes_pessoais")
	informacoes_profissionais = applicants.get(id).get("informacoes_profissionais")
	formacao_e_idiomas = applicants.get(id).get("formacao_e_idiomas")
	aux = {
		'id': int(id),
		'pcd': informacoes_pessoais.get('pcd'),
		'titulo_profissional': informacoes_profissionais.get('titulo_profissional'),
		'area_atuacao': informacoes_profissionais.get('area_atuacao'),
		'conhecimentos_tecnicos': informacoes_profissionais.get('conhecimentos_tecnicos'),
		'certificacoes': informacoes_profissionais.get('certificacoes'),
		'outras_certificacoes': informacoes_profissionais.get('outras_certificacoes'),
		'nivel_profissional': informacoes_profissionais.get('nivel_profissional'),
		'nivel_academico': formacao_e_idiomas.get('nivel_academico'),
		'nivel_ingles': formacao_e_idiomas.get('nivel_ingles'),
		'nivel_espanhol': formacao_e_idiomas.get('nivel_espanhol'),
		'outro_idioma': formacao_e_idiomas.get('outro_idioma'),
		'cv_pt': applicants.get(id).get("cv_pt"),
	}
	rows.append(aux)

applicants_df = pd.DataFrame(data=rows, columns=['id', 'pcd', 'endereco', 'titulo_profissional', 'area_atuacao', 'conhecimentos_tecnicos','certificacoes', 'outras_certificacoes', 'nivel_profissional','nivel_academico', 'nivel_ingles', 'nivel_espanhol', 'outro_idioma', 'cv_pt'])

In [ ]:
with open(Data_path + 'vagas.json', 'r', encoding='utf-8') as file:
	vagas = json.load(file)

rows = []
for vaga in vagas:
	id = vaga
	perfil_vaga = vagas.get(id).get("perfil_vaga")
	aux = {
		'id': int(id),
		'vaga_especifica_para_pcd': perfil_vaga.get('vaga_especifica_para_pcd'),
		'nivel_profissional': perfil_vaga.get('nivel profissional'),
		'nivel_academico': perfil_vaga.get('nivel_academico'),
		'nivel_ingles': perfil_vaga.get('nivel_ingles'),
		'nivel_espanhol': perfil_vaga.get('nivel_espanhol'),
		'outro_idioma': perfil_vaga.get('outro_idioma'),
		'areas_atuacao': perfil_vaga.get('areas_atuacao'),
		'principais_atividades': perfil_vaga.get('principais_atividades'),
		'demais_observacoes': perfil_vaga.get('demais_observacoes'),
		'viagens_requeridas': perfil_vaga.get('viagens_requeridas')
	}
	rows.append(aux)

vagas_df = pd.DataFrame(data=rows, columns=['id', 'estado', 'vaga_especifica_para_pcd', 'nivel_profissional', 'nivel_academico', 'nivel_ingles', 'nivel_espanhol', 'outro_idioma', 'areas_atuacao', 'principais_atividades', 'demais_observacoes', 'viagens_requeridas'])

In [ ]:
with open(Data_path + 'prospects.json', 'r', encoding='utf-8') as file:
	prospects = json.load(file)

rows = []
for prospect in prospects:
	id = prospect
	candidates = prospects.get(id).get("prospects")
	for candidate in candidates:
		aux_id:int = int(id)
		aux = {
			'id_vaga': aux_id,
			'id_applicant': int(candidate.get("codigo")),
			'situacao_candidado': candidate.get("situacao_candidado"),
			'comentario': candidate.get("comentario")
		}
		rows.append(aux)

prospects_df = pd.DataFrame(data=rows, columns=['id_vaga','id_applicant', 'situacao_candidado', 'comentario'])

In [ ]:
aux_merge_ap = pd.merge(applicants_df, prospects_df, left_on='id', right_on='id_applicant', how='inner')
merge = pd.merge(aux_merge_ap, vagas_df, left_on='id_vaga', right_on='id', how='inner', suffixes=('', '_vaga'))

df = merge[[
    'pcd',
    'titulo_profissional',
    'area_atuacao',
    'conhecimentos_tecnicos',
    'certificacoes',
    'outras_certificacoes',
    'nivel_academico',
    'nivel_profissional',
    'nivel_ingles',
    'nivel_espanhol',
    'outro_idioma',
    'cv_pt',
    'situacao_candidado',
    'comentario',
    'vaga_especifica_para_pcd',
    'nivel_profissional_vaga',
    'nivel_academico_vaga',
    'nivel_ingles_vaga',
    'nivel_espanhol_vaga',
    'outro_idioma_vaga',
    'areas_atuacao',
    'principais_atividades',
    'demais_observacoes',
    'viagens_requeridas'
]]

df = df.drop_duplicates()

In [ ]:
del applicants
del vagas
del prospects
del applicants_df
del prospects_df
del vagas_df
del aux_merge_ap
del merge
gc.collect()

## Data Pre Processor

In [ ]:
def string_normalizer(text):
	if text is None or text == "":
		return None
	else:
		return ''.join(c for c in unicodedata.normalize('NFD', text) if not unicodedata.combining(c)).lower()

In [ ]:
nivel_map = {
    'nenhum': 0,
    'basico': 1,
    'intermediario': 2,
    'avancado': 3,
    'fluente': 4
}

In [ ]:
df = df.apply(lambda col: col.map(string_normalizer) if col.dtype == 'object' else col)
df.replace('-', None, inplace=True)

In [ ]:
df['pcd'] = df['pcd'].apply(lambda x: 1 if x == 'sim' else 0)
df['vaga_especifica_para_pcd'] = df['vaga_especifica_para_pcd'].apply(lambda x: 1 if x == 'sim' else 0)
df['pcd_match'] = ((df['pcd'] == 1) & (df['vaga_especifica_para_pcd'] == 1)).astype(int)

In [ ]:
df['viagens_requeridas'] = df['viagens_requeridas'].apply(lambda x: 1 if x == 'sim' else 0)

In [ ]:
df['nivel_ingles'] = df['nivel_ingles'].map(nivel_map).fillna(0)
df['nivel_ingles_vaga'] = df['nivel_ingles_vaga'].map(nivel_map).fillna(0)
df['nivel_espanhol'] = df['nivel_ingles'].map(nivel_map).fillna(0)
df['nivel_espanhol_vaga'] = df['nivel_ingles_vaga'].map(nivel_map).fillna(0)

df['ingles'] = (df['nivel_ingles'] >= df['nivel_ingles_vaga']).astype(int)
df['espanhol'] = (df['nivel_espanhol'] >= df['nivel_espanhol_vaga']).astype(int)

In [ ]:
df.drop(columns=['pcd', 'vaga_especifica_para_pcd', 'nivel_ingles', 'nivel_ingles_vaga', 'nivel_espanhol','nivel_espanhol_vaga'], inplace=True)

df.rename(columns={
    'titulo_profissional': 'candidato_titulo_profissional',
    'area_atuacao': 'candidato_area_atuacao',
    'conhecimentos_tecnicos': 'candidato_conhecimentos_tecnicos',
    'certificacoes': 'candidato_certificacoes',
    'outras_certificacoes': 'candidato_outras_certificacoes',
    'nivel_academico': 'candidato_nivel_academico',
    'nivel_profissional': 'candidato_nivel_profissional',
    'outro_idioma': 'candidato_outro_idioma',
    'cv_pt': 'candidato_cv',
    'situacao_candidado': 'situacao',
    'comentario': 'comentario',
    'nivel_profissional_vaga': 'vaga_nivel_profissional',
    'nivel_academico_vaga': 'vaga_nivel_academico',
    'outro_idioma_vaga': 'vaga_outro_idioma',
    'areas_atuacao': 'vaga_area_atuacao',
    'principais_atividades': 'vaga_principais_atividades',
    'demais_observacoes': 'vaga_demais_observacoes',
    'viagens_requeridas': 'vaga_viagens_requeridas',
    'pcd_match': 'pcd',
    'ingles': 'nivel_ingles',
    'espanhol': 'nivel_espanhol'
}, inplace=True)

In [ ]:
print(df.shape)
print(df.info())

## Model

## Train

## Predict

## Metrics